# Data processing to get cases, deaths, hospitalizations, change in hospitalizations, and admissions

In [181]:
#%reset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 


hfont = {'fontname':'Helvetica'}

In [182]:
def data_prep_dates(dataset, columns, date_column, start_date, number_weeks, is_weeks):
    """
    Takes data frame with daily or weekly data and subsets rows between two dates. 
    """

    dataset = dataset[columns]

    dataset = dataset.sort_values(date_column).reset_index(drop=True)

    dataset['index'] = range(len(dataset[date_column]))
    start_date_row = dataset[dataset[date_column] == start_date]['index'].min()

    start_date_index_dates = int(np.where(dataset[date_column].unique() == start_date)[0])
    if is_weeks: 
        six_month_date_index_dates = dataset[date_column].unique()[number_weeks + start_date_index_dates]
    else:
            six_month_date_index_dates = dataset[date_column].unique()[(number_weeks * 7) + start_date_index_dates]

    end_date_row = dataset[dataset[date_column] == six_month_date_index_dates]['index'].max()
    dataset = dataset[start_date_row:end_date_row]
    return(dataset)

def weekly_date_range(dataset):
    dataset['date'] = pd.to_datetime(dataset['date'])

    start_date = dataset['date'].min()
    end_date = dataset['date'].max()
    dates = []
    current_date = start_date

    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=7)
    return dates

def convert_state_name_to_abbreviation(state_name):
    state_name = state_name.lower().strip()

    state_abbreviations = {
        'alabama': 'AL',
        'alaska': 'AK',
        'arizona': 'AZ',
        'arkansas': 'AR',
        'california': 'CA',
        'colorado': 'CO',
        'connecticut': 'CT',
        'delaware': 'DE',
        'florida': 'FL',
        'georgia': 'GA',
        'hawaii': 'HI',
        'idaho': 'ID',
        'illinois': 'IL',
        'indiana': 'IN',
        'iowa': 'IA',
        'kansas': 'KS',
        'kentucky': 'KY',
        'louisiana': 'LA',
        'maine': 'ME',
        'maryland': 'MD',
        'massachusetts': 'MA',
        'michigan': 'MI',
        'minnesota': 'MN',
        'mississippi': 'MS',
        'missouri': 'MO',
        'montana': 'MT',
        'nebraska': 'NE',
        'nevada': 'NV',
        'new hampshire': 'NH',
        'new jersey': 'NJ',
        'new mexico': 'NM',
        'new york': 'NY',
        'north carolina': 'NC',
        'north dakota': 'ND',
        'ohio': 'OH',
        'oklahoma': 'OK',
        'oregon': 'OR',
        'pennsylvania': 'PA',
        'rhode island': 'RI',
        'south carolina': 'SC',
        'south dakota': 'SD',
        'tennessee': 'TN',
        'texas': 'TX',
        'utah': 'UT',
        'vermont': 'VT',
        'virginia': 'VA',
        'washington': 'WA',
        'washington dc': 'DC',
        'west virginia': 'WV',
        'wisconsin': 'WI',
        'wyoming': 'WY',
        'district of columbia': 'DC'
    }

    return state_abbreviations.get(state_name, None)

def convert_daily_weekly(dataset, column_name, date_column, geography_column):
    dataset[date_column] = pd.to_datetime(dataset[date_column])

    dates = weekly_date_range(dataset)

    num_rows = len(dates)*len(dataset[geography_column].unique())
    weekly_dataframe = pd.DataFrame(columns=dataset.columns[range(len(column_name))], index=range(num_rows))
    weekly_dataframe.columns = column_name

    x = -1
    for geography in dataset[geography_column].unique():
        state_data = dataset[dataset[geography_column] == geography].reset_index()
        for date in dates: 
            x += 1
            weekly_dataframe.iloc[x,1] = geography
            weekly_dataframe.iloc[x,0] = date
            if (state_data.loc[0, 'date'] < date) | (state_data.loc[0, 'date'] <= (date - timedelta(days=7))):
                end_index = state_data.loc[state_data['date'] < date, 'date'].idxmax()
                start_index = state_data.loc[(state_data['date'] <= (date - timedelta(days=7))), 'date'].idxmax()
                selected_rows = state_data.iloc[start_index:end_index, len(state_data.columns)-1]
                cumulative_sum = selected_rows.sum()
                weekly_dataframe.iloc[x,2] = cumulative_sum

    return(weekly_dataframe)

def per_100k(dataset, date_column, value_column, geography_column, categories_to_create, populations, hospitalizations, threshold ):
    geography_names = dataset[geography_column].unique()
    dataset = dataset.pivot_table(index= date_column, columns=geography_column, values=value_column) # gets rid of week - 2 
    dataset = dataset.reset_index()
    for geography in geography_names: 
        for column in categories_to_create:
            col_name_rate = geography + column
            dataset[col_name_rate] = dataset[geography]/populations[geography] * 100000
            col_name_delta = geography + '_delta_100k'
            j = 0
            for row in range(len(dataset[geography]) - 1): ## need to use j as an index as row is a datetime object 
                    if(j != 0):
                        dataset.loc[j, col_name_delta] = dataset.loc[j, col_name_rate] - dataset.loc[j - 1, col_name_rate]
                    j+=1
    if hospitalizations: 
            col_name_threshold = geography + '_over_' + str(threshold) + '_100k'
            dataset[col_name_threshold] = (dataset[col_name_rate] > threshold)*1
    # remove first row with week - 1
    dataset = dataset[dataset[date_column] != dataset.loc[0,date_column] ] ## remove week 0 

    return(dataset)

In [183]:
population_by_state_April_2020 = {
    'Alabama': 5024279,
    'Alaska': 733391,
    'Arizona': 7151502,
    'Arkansas': 3011524,
    'California': 39538223,
    'Colorado': 5773714,
    'Connecticut': 3605944,
    'Delaware': 989948,
    'District of Columbia': 689545,
    'Florida': 21538187,
    'Georgia': 10711908,
    'Hawaii': 1455271,
    'Idaho': 1839106,
    'Illinois': 12812508,
    'Indiana': 6785528,
    'Iowa': 3190369,
    'Kansas': 2937880,
    'Kentucky': 4505836,
    'Louisiana': 4657757,
    'Maine': 1362359,
    'Maryland': 6177224,
    'Massachusetts': 7029917,
    'Michigan': 10077331,
    'Minnesota': 5706494,
    'Mississippi': 2961279,
    'Missouri': 6154913,
    'Montana': 1084225,
    'Nebraska': 1961504,
    'Nevada': 3104614,
    'New Hampshire': 1377529,
    'New Jersey': 9288994,
    'New Mexico': 2117522,
    'New York': 20201249,
    'North Carolina': 10439388,
    'North Dakota': 779094,
    'Ohio': 11799448,
    'Oklahoma': 3959353,
    'Oregon': 4237256,
    'Pennsylvania': 13002700,
    'Rhode Island': 1097379,
    'South Carolina': 5118425,
    'South Dakota': 886667,
    'Tennessee': 6910840,
    'Texas': 29145505,
    'Utah': 3271616,
    'Vermont': 643077,
    'Virginia': 8631393,
    'Washington': 7705281,
    'West Virginia': 1793716,
    'Wisconsin': 5893718,
    'Wyoming': 576851
}
population_by_state_April_2020_abb = {}
states_to_remove = []

for state_name in population_by_state_April_2020:
    state_abbreviation = convert_state_name_to_abbreviation(state_name)
    if state_abbreviation:
        population_by_state_April_2020_abb[state_abbreviation] = population_by_state_April_2020[state_name]




## Data from https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh
- Hospitalizations
- Use total adult patients with covid and total pediatric patients with covid for total covid cases
- Try and plot to match up 
- Also has previous day admissions - can try and get cumulative over 7 days to match with weekly admissions

In [184]:
State_hospitalizations = pd.read_csv("/Users/rem76/Documents/COVID_projections/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW_.csv")
# Remove non-states (except DC)

State_hospitalizations = State_hospitalizations[(State_hospitalizations['state'] != 'PR') & (State_hospitalizations['state'] != 'VI') & (State_hospitalizations['state'] != 'GU')& (State_hospitalizations['state'] != 'AS')& (State_hospitalizations['state'] != 'MP')]


Overall hospital numbers

In [185]:
State_hospitalizations_total_hospitalizations = data_prep_dates(State_hospitalizations, columns = ['date', 'state', 'percent_of_inpatients_with_covid_numerator'], date_column = 'date', start_date = '2020/06/21', number_weeks = 26, is_weeks = False)
## Start date is two week before the actual date of interest, allows us to calculate the weekly and change in weekly rate for our "actual" first week 

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45414/18807921.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  start_date_index_dates = int(np.where(dataset[date_column].unique() == start_date)[0])


In [186]:
State_hospitalizations_total_hospitalizations = State_hospitalizations_total_hospitalizations[['date', 'state', 'percent_of_inpatients_with_covid_numerator']]
State_hospitalizations_total_hospitalizations.rename(columns={'percent_of_inpatients_with_covid_numerator': 'case_numbers'}, inplace=True)


In [187]:
State_hospitalizations_total_hospitalizations_weekly = convert_daily_weekly(State_hospitalizations_total_hospitalizations, ['date', 'state', 'case_numbers'], 'date', 'state')


In [188]:
State_hospitalizations_total_hospitalizations_weekly = per_100k(State_hospitalizations_total_hospitalizations_weekly, date_column = 'date', value_column = 'case_numbers', geography_column = 'state', categories_to_create = ['hospitalizations'],populations =  population_by_state_April_2020_abb, hospitalizations = True, threshold = 15 )

/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45414/18807921.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[col_name_rate] = dataset[geography]/populations[geography] * 100000
/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45414/18807921.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset.loc[j, col_name_delta] = dataset.loc[j, col_name_rate] - dataset.loc[j - 1, col_name_rate]
/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_45414/18807921.py:128: PerformanceWarning: D